# Import packages

In [1]:
%matplotlib inline
import qiime2, biom, pickle
import pandas as pd
import numpy as np
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, \
                            metadata, feature_table, alignment, \
                            phylogeny, diversity, emperor, feature_classifier, \
                            taxa, composition

## Modify settings

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

# Loading Patient Pickled Metadata

In [3]:
# Load pickle files to demonstrate that the Pandas formatting is not lost in the binary conversions
caseTesting = pd.read_pickle(open("caseGroupedDict.p", "rb"))
fileNameTesting = pd.read_pickle(open("fileGroupedDict.p", "rb"))

In [4]:
# Example of loading Pickle file by case key (ie per patient)
caseTestingKey = '7861a5f2-8910-4bcb-9c34-f79c5acd6e21'.upper()
caseTesting[caseTestingKey]

,ageAtDiagnosis,aliquot_name,case_name,clinical_m_label,clinical_n_label,clinical_t_label,data_format_label,data_type_label,disease_type_label,ethnicity_label,experimental_strategy_label,file_name,file_submitter_id,gdc_file_uuid,gender_label,histological_diagnosis_label,pathologic_m_label,pathologic_n_label,pathologic_stage_label,pathologic_t_label,perf_score_eastern_cancer_oncology_group_label,perf_score_karnofsky_label,primary_site_label,prior_dx,race_label,ref_genome_label,sample_label,sample_type_label,aliquot_concentration,analyte_A260A280Ratio,analyte_amount,analyte_type_label,analyte_well_number,country_of_sample_procurement,data_submitting_center_label,days_to_last_followup,file_last_modified_date,file_published_date,file_upload_date,freezing_method_label,icd03_histology_label,icd03_histology_site,icd10,new_tumor_event_after_initial_trtmt,new_tumor_event_label,perf_score_timing_label,portion_is_ffpe,portion_number,portion_slide_label,portion_weight,primary_therapy_outcome_success_label,radiation_therapy_code_label,radiation_therapy_site_label,radiation_therapy_type_label,seq_platform_label,spectrophotometer_method_label,tissue_source_site_label,vital_status_label,year_of_diagnosis
42,42,1CCF908A-7620-4AF8-B83F-0E287F3655C5,7861A5F2-8910-4BCB-9C34-F79C5ACD6E21,Not available,Not available,Not available,BAM,Raw sequencing data,Breast Invasive Carcinoma,Not available,RNA-Seq,UNCID_1147830.1ccf908a-7620-4af8-b83f-0e287f3655c5.sorted_genome_alignments.bam,D38DF7DB-3C58-4865-AE5A-E1E0B80201F1,C0D7AB92-46B4-4940-AEFF-881241D7E017,FEMALE,Infiltrating Ductal Carcinoma,Not available,N1a,Stage IIB,T2,Not available,Not available,Breast,No,Not available,HG19,D3263082-946D-4420-8A63-8D67EEEE09C5,Primary Tumor,0.15,1.79,59.37,RNA,8EA8BD47-0E8A-4B6F-B87A-27C1BDEF9B7E,Germany,University of North Carolina,518,2013-05-16T20:56:17.000Z,2012-06-21T22:01:14.000Z,2012-05-15T15:00:21.000Z,Not available,8500/3,C50.9,C50.9,NaN,NaN,Not available,NO,11,3658536C-C029-4685-979F-11E914E53E4E,30,Not available,7D9218C9-489E-4C64-9C92-7EE1AEB9C75E,Primary Tumor Field,External,Illumina HiSeq,UV Spec,Indivumed,Alive,2009
43,42,1CCF908A-7620-4AF8-B83F-0E287F3655C5,7861A5F2-8910-4BCB-9C34-F79C5ACD6E21,Not available,Not available,Not available,BAM,Raw sequencing data,Breast Invasive Carcinoma,Not available,RNA-Seq,UNCID_1147830.1ccf908a-7620-4af8-b83f-0e287f3655c5.sorted_genome_alignments.bam,D38DF7DB-3C58-4865-AE5A-E1E0B80201F1,C0D7AB92-46B4-4940-AEFF-881241D7E017,FEMALE,Infiltrating Ductal Carcinoma,Not available,N1a,Stage IIB,T2,Not available,Not available,Breast,No,Not available,HG19,D3263082-946D-4420-8A63-8D67EEEE09C5,Primary Tumor,0.15,1.79,59.37,RNA,8EA8BD47-0E8A-4B6F-B87A-27C1BDEF9B7E,Germany,University of North Carolina,518,2013-05-16T20:56:17.000Z,2012-06-21T22:01:14.000Z,2012-05-15T15:00:21.000Z,Not available,8500/3,C50.9,C50.9,NaN,NaN,Not available,NO,11,F15749FB-E5FD-4124-AD85-EEC12751816B,30,Not available,7D9218C9-489E-4C64-9C92-7EE1AEB9C75E,Primary Tumor Field,External,Illumina HiSeq,UV Spec,Indivumed,Alive,2009
44,42,6C8E9197-FD16-4FED-BFDA-E349CAB26314,7861A5F2-8910-4BCB-9C34-F79C5ACD6E21,Not available,Not available,Not available,BAM,Raw sequencing data,Breast Invasive Carcinoma,Not available,WGS,0d9ef8b814476b334be08fe7ee6ea88c.bam,8244953D-EBE8-4A88-A547-00185B4CB18A,4D604B9E-E0AE-4C4E-B844-4C1D7924D75B,FEMALE,Infiltrating Ductal Carcinoma,Not available,N1a,Stage IIB,T2,Not available,Not available,Breast,No,Not available,GRCh37-lite,D3263082-946D-4420-8A63-8D67EEEE09C5,Primary Tumor,0.07,2.23,66.73,DNA,604D99A4-3F8E-4267-BB83-C1EF582899C3,Germany,Washington University School of Medicine,518,2014-06-19T01:26:13.000Z,2012-05-25T02:12:55.000Z,2012-05-24T14:30:44.000Z,Not available,8500/3,C50.9,C50.9,NaN,NaN,Not available,NO,11,3658536C-C029-4685-979F-11E914E53E4E,30,Not available,7D9218C9-489E-4C64-9C92-7EE1AEB9C75E,Primary Tumor Field,External,Illumina HiSeq,UV Spec,Indivumed,Alive,2009
45,42,6C8E9197-FD16-4FED-BFDA-E349CAB26314,7861A5F2-89

In [5]:
# Example of loading Pickle file by file key (ie per sample)
fileTestingKey = '01249B8C-2E9E-4CEA-B39A-485863AB231A'
fileNameTesting[fileTestingKey]

,ageAtDiagnosis,aliquot_name,case_name,clinical_m_label,clinical_n_label,clinical_t_label,data_format_label,data_type_label,disease_type_label,ethnicity_label,experimental_strategy_label,file_name,file_submitter_id,gdc_file_uuid,gender_label,histological_diagnosis_label,pathologic_m_label,pathologic_n_label,pathologic_stage_label,pathologic_t_label,perf_score_eastern_cancer_oncology_group_label,perf_score_karnofsky_label,primary_site_label,prior_dx,race_label,ref_genome_label,sample_label,sample_type_label,aliquot_concentration,analyte_A260A280Ratio,analyte_amount,analyte_type_label,analyte_well_number,country_of_sample_procurement,data_submitting_center_label,days_to_last_followup,file_last_modified_date,file_published_date,file_upload_date,freezing_method_label,icd03_histology_label,icd03_histology_site,icd10,new_tumor_event_after_initial_trtmt,new_tumor_event_label,perf_score_timing_label,portion_is_ffpe,portion_number,portion_slide_label,portion_weight,primary_therapy_outcome_success_label,radiation_therapy_code_label,radiation_therapy_site_label,radiation_therapy_type_label,seq_platform_label,spectrophotometer_method_label,tissue_source_site_label,vital_status_label,year_of_diagnosis
5070,64,9ABEF075-5168-4A66-9826-FD61C94B8F6D,C83D38FC-E011-4F75-A100-96513611F3E9,Not available,Not available,Not available,BAM,Raw sequencing data,Colon Adenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,UNCID_2153922.9abef075-5168-4a66-9826-fd61c94b8f6d.sorted_genome_alignments.bam,39C04C87-8662-4704-9E72-53F94D852914,01249B8C-2E9E-4CEA-B39A-485863AB231A,FEMALE,Colon Adenocarcinoma,Not available,N1a,Stage IIIC,T4b,Not available,Not available,Colorectal,No,BLACK OR AFRICAN AMERICAN,HG19,69EF3E70-5D7A-4E30-ABDE-0F9F76EA3141,Primary Tumor,0.17,2.23,24.87,RNA,02451886-80DE-42DE-885F-5A8BBF98016D,United States,University of North Carolina,821,2013-08-28T22:31:06.000Z,2013-08-28T22:31:06.000Z,2013-08-28T22:15:17.000Z,Not available,8140/3,C18.7,C18.7,NO,NaN,Not available,NO,11,B81997A0-3519-4647-9CBB-A8E62A31E839,30,Complete Remission/Response,NaN,NaN,NaN,Illumina HiSeq,UV Spec,BLN - Baylor,Alive,2011


# Loading Cancer Microbiome Data

In [6]:
workdir = '/Users/gdpoore/Google Drive/AAA_UCSD_Research/AAA_Knight_Lab/AAA_Projects/AAA_TCGA/All_Tumor_Analysis_GDP082118/'
# %cd $workdir

In [7]:
bactDataBarn = biom.load_table(workdir+'Bacterial_all_cancer_types.biom')
virDataBarn = biom.load_table(workdir+'Viral_all_cancer_types.biom')

# Convert BIOM format to QIIME format
bactDataBarnQ2 = qiime2.Artifact.import_data('FeatureTable[Frequency]', bactDataBarn)
virDataBarnQ2 = qiime2.Artifact.import_data('FeatureTable[Frequency]', virDataBarn)

In [8]:
bactDataBarnDF = bactDataBarnQ2.view(pd.DataFrame)
virDataBarnDF = virDataBarnQ2.view(pd.DataFrame)

In [9]:
print(bactDataBarnDF.shape)
print(virDataBarnDF.shape)
bactDataBarnDF.head()
virDataBarnDF.head()

(18124, 1626)
(18120, 367)


,k__Viruses;f__Phycodnaviridae;g__Prasinovirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Sfi1unalikevirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Simplexvirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__C2likevirus,k__Viruses;f__Poxviridae;g__Parapoxvirus,k__Viruses;f__Papillomaviridae;g__Dyolambdapapillomavirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Bignuzlikevirus,k__Viruses;o__Mononegavirales;f__Bornaviridae;g__Bornavirus,k__Viruses;f__Bicaudaviridae;g__Bicaudavirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Aquamavirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Scutavirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Mardivirus,k__Viruses;f__Baculoviridae;g__Deltabaculovirus,k__Viruses;f__Papillomaviridae;g__Taupapillomavirus,k__Viruses;o__Caudovirales;f__Myoviridae;g__I3likevirus,k__Viruses;f__Virgaviridae;g__Furovirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Proboscivirus,k__Viruses;o__Herpesvirales;f__Alloherpesviridae;g__Ictalurivirus,k__Viruses;f__Inoviridae;g__Inovirus,k__Viruses;f__Marseilleviridae;g__Marseillevirus,k__Viruses;o__Nidovirales;f__Coronaviridae;g__Gammacoronavirus,k__Viruses;f__Anelloviridae;g__Thetatorquevirus,k__Viruses;f__Ascoviridae;g__Ascovirus,k__Viruses;o__Tymovirales;f__Tymoviridae;g__Maculavirus,k__Viruses;g__Negevirus,k__Viruses;o__Caudovirales;f__Myoviridae;g__Spo1virus,k__Viruses;f__Circoviridae;g__Gyrovirus,k__Viruses;o__Mononegavirales;f__Rhabdoviridae;g__Novirhabdovirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Sicinivirus,k__Viruses;o__Mononegavirales;g__Pneumovirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Dicipivirus,k__Viruses;f__Poxviridae;g__Molluscipoxvirus,k__Viruses;o__Tymovirales;f__Tymoviridae;g__Tymovirus,k__Viruses;f__Arenaviridae;g__Mammarenavirus,k__Viruses;f__Totiviridae;g__Victorivirus,k__Viruses;f__Flaviviridae;g__Hepacivirus,k__Viruses;f__Poxviridae;g__Alphaentomopoxvirus,k__Viruses;f__Anelloviridae;g__Gammatorquevirus,k__Viruses;o__Tymovirales;f__Gammaflexiviridae;g__Mycoflexivirus,k__Viruses;g__Tenuivirus,k__Viruses;f__Hepeviridae;g__Piscihepevirus,k__Viruses;f__Virgaviridae;g__Pomovirus,k__Viruses;f__Caulimoviridae;g__Badnavirus,k__Viruses;f__Polydnaviridae;g__Ichnovirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Percavirus,k__Viruses;f__Papillomaviridae;g__Betapapillomavirus,k__Viruses;f__Togaviridae;g__Alphavirus,k__Viruses;o__Tymovirales;f__Betaflexiviridae;g__Trichovirus,k__Viruses;o__Caudovirales;f__Podoviridae;g__Bcep22likevirus,k__Viruses;o__Tymovirales;f__Alphaflexiviridae;g__Sclerodarnavirus,k__Viruses;f__Phycodnaviridae;g__Coccolithovirus,k__Viruses;o__Ligamenvirales;f__Lipothrixviridae;g__Deltalipothrixvirus,k__Viruses;f__Papillomaviridae;g__Omikronpapillomavirus,k__Viruses;f__Retroviridae;g__Alpharetrovirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Jerseylikevirus,k__Viruses;f__Poxviridae;g__Capripoxvirus,k__Viruses;f__Poxviridae;g__Cervidpoxvirus,k__Viruses;g__Higrevirus,k__Viruses;f__Potyviridae;g__Bymovirus,k__Viruses;o__Caudovirales;f__Podoviridae;g__Phikmvlikevirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Tremovirus,k__Viruses;o__Picornavirales;f__Secoviridae;g__Torradovirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__T5likevirus,k__Viruses;o__Nidovirales;f__Coronaviridae;g__Bafinivirus,k__Viruses;f__Baculoviridae;g__Alphabaculovirus,k__Viruses;f__Papillomaviridae;g__Mupapillomavirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Lambdalikevirus,k__Viruses;o__Caudovirales;f__Myoviridae;g__Felixounalikevirus,k__Viruses;f__Papillomaviridae;g__Phipapillomavirus,k__Viruses;f__Papillomaviridae;g__Omegapapillomavirus,k__Viruses;o__Nidovirales;f__Coronaviridae;g__Betacoronavirus,k__Viruses;f__Luteoviridae;g__Enamovirus,k__Viruses;f__Phycodnaviridae;g__Chlorovirus,k__Viruses;f__Hypoviridae;g__Hypovirus,k__Viruses;f__Anelloviridae;g__Alphatorquevirus,k__Viruses;o__Picornavirales;f__Dicistroviridae;g__Aparavirus,k__Viruses;g__Pithovirus,k__Viruses;f__Phycodnaviridae;g__Phaeovirus,k__

In [10]:
vbDataBarnDF = pd.merge(virDataBarnDF, bactDataBarnDF, how='left', left_index=True, right_index=True)
print(vbDataBarnDF.shape) # Want this to be 18120 x 1993
vbDataBarnDF.head()

(18120, 1993)


,k__Viruses;f__Phycodnaviridae;g__Prasinovirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Sfi1unalikevirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Simplexvirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__C2likevirus,k__Viruses;f__Poxviridae;g__Parapoxvirus,k__Viruses;f__Papillomaviridae;g__Dyolambdapapillomavirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Bignuzlikevirus,k__Viruses;o__Mononegavirales;f__Bornaviridae;g__Bornavirus,k__Viruses;f__Bicaudaviridae;g__Bicaudavirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Aquamavirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Scutavirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Mardivirus,k__Viruses;f__Baculoviridae;g__Deltabaculovirus,k__Viruses;f__Papillomaviridae;g__Taupapillomavirus,k__Viruses;o__Caudovirales;f__Myoviridae;g__I3likevirus,k__Viruses;f__Virgaviridae;g__Furovirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Proboscivirus,k__Viruses;o__Herpesvirales;f__Alloherpesviridae;g__Ictalurivirus,k__Viruses;f__Inoviridae;g__Inovirus,k__Viruses;f__Marseilleviridae;g__Marseillevirus,k__Viruses;o__Nidovirales;f__Coronaviridae;g__Gammacoronavirus,k__Viruses;f__Anelloviridae;g__Thetatorquevirus,k__Viruses;f__Ascoviridae;g__Ascovirus,k__Viruses;o__Tymovirales;f__Tymoviridae;g__Maculavirus,k__Viruses;g__Negevirus,k__Viruses;o__Caudovirales;f__Myoviridae;g__Spo1virus,k__Viruses;f__Circoviridae;g__Gyrovirus,k__Viruses;o__Mononegavirales;f__Rhabdoviridae;g__Novirhabdovirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Sicinivirus,k__Viruses;o__Mononegavirales;g__Pneumovirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Dicipivirus,k__Viruses;f__Poxviridae;g__Molluscipoxvirus,k__Viruses;o__Tymovirales;f__Tymoviridae;g__Tymovirus,k__Viruses;f__Arenaviridae;g__Mammarenavirus,k__Viruses;f__Totiviridae;g__Victorivirus,k__Viruses;f__Flaviviridae;g__Hepacivirus,k__Viruses;f__Poxviridae;g__Alphaentomopoxvirus,k__Viruses;f__Anelloviridae;g__Gammatorquevirus,k__Viruses;o__Tymovirales;f__Gammaflexiviridae;g__Mycoflexivirus,k__Viruses;g__Tenuivirus,k__Viruses;f__Hepeviridae;g__Piscihepevirus,k__Viruses;f__Virgaviridae;g__Pomovirus,k__Viruses;f__Caulimoviridae;g__Badnavirus,k__Viruses;f__Polydnaviridae;g__Ichnovirus,k__Viruses;o__Herpesvirales;f__Herpesviridae;g__Percavirus,k__Viruses;f__Papillomaviridae;g__Betapapillomavirus,k__Viruses;f__Togaviridae;g__Alphavirus,k__Viruses;o__Tymovirales;f__Betaflexiviridae;g__Trichovirus,k__Viruses;o__Caudovirales;f__Podoviridae;g__Bcep22likevirus,k__Viruses;o__Tymovirales;f__Alphaflexiviridae;g__Sclerodarnavirus,k__Viruses;f__Phycodnaviridae;g__Coccolithovirus,k__Viruses;o__Ligamenvirales;f__Lipothrixviridae;g__Deltalipothrixvirus,k__Viruses;f__Papillomaviridae;g__Omikronpapillomavirus,k__Viruses;f__Retroviridae;g__Alpharetrovirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Jerseylikevirus,k__Viruses;f__Poxviridae;g__Capripoxvirus,k__Viruses;f__Poxviridae;g__Cervidpoxvirus,k__Viruses;g__Higrevirus,k__Viruses;f__Potyviridae;g__Bymovirus,k__Viruses;o__Caudovirales;f__Podoviridae;g__Phikmvlikevirus,k__Viruses;o__Picornavirales;f__Picornaviridae;g__Tremovirus,k__Viruses;o__Picornavirales;f__Secoviridae;g__Torradovirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__T5likevirus,k__Viruses;o__Nidovirales;f__Coronaviridae;g__Bafinivirus,k__Viruses;f__Baculoviridae;g__Alphabaculovirus,k__Viruses;f__Papillomaviridae;g__Mupapillomavirus,k__Viruses;o__Caudovirales;f__Siphoviridae;g__Lambdalikevirus,k__Viruses;o__Caudovirales;f__Myoviridae;g__Felixounalikevirus,k__Viruses;f__Papillomaviridae;g__Phipapillomavirus,k__Viruses;f__Papillomaviridae;g__Omegapapillomavirus,k__Viruses;o__Nidovirales;f__Coronaviridae;g__Betacoronavirus,k__Viruses;f__Luteoviridae;g__Enamovirus,k__Viruses;f__Phycodnaviridae;g__Chlorovirus,k__Viruses;f__Hypoviridae;g__Hypovirus,k__Viruses;f__Anelloviridae;g__Alphatorquevirus,k__Viruses;o__Picornavirales;f__Dicistroviridae;g__Aparavirus,k__Viruses;g__Pithovirus,k__Viruses;f__Phycodnaviridae;g__Phaeovirus,k__

# Loading QIIME Metadata File

In [11]:
sample_metadata = qiime2.Metadata.load(workdir+'tcga_mapping_file_updated082118.tsv')
dfMeta = sample_metadata.to_dataframe()
print(dfMeta.shape)
dfMeta.head()

(18150, 27)


,BarcodeSequence,LinkerPrimerSequence,filename,filenames_new_suffix,age_at_diagnosis,aliquot_id,aliquot_uuid,case_id,case_uuid,data_format,data_subtype,data_type,days_to_death,disease_type,ethnicity,experimental_strategy,gender,investigation,platform,primary_site,race,reference_genome,sample_id,sample_type,sample_uuid,vital_status,Description
#SampleID,,,,,,,,,,,,,,,,,,,,,,,,,,,
s10247,NaN,NaN,G14083.TCGA-21-1082-01A-01D-1521-08.3.bam,G14083.TCGA-21-1082-01A-01D-1521-08.3.fasta,61,TCGA-21-1082-01A-01D-1521-08,1C3DF485-8E75-4378-87F6-C6463A520624,TCGA-21-1082,0e2ee54a-51c9-4868-842d-a2a1c1cfb016,BAM,Aligned reads,Raw sequencing data,NA,Lung Squamous Cell Carcinoma,NOT HISPANIC OR LATINO,WGS,MALE,TCGA-LUSC,Illumina HiSeq,Lung,WHITE,HG19_Broad_variant,TCGA-21-1082-01A,Primary Tumor,F091C76E-5D01-4F4B-8F6F-463C9F302930,Alive,s361__G14083.TCGA-21-1082-01A-01D-1521-08.3.bam
s10248,NaN,NaN,G13226.TCGA-21-1078-01A-01D-1521-08.1.bam,G13226.TCGA-21-1078-01A-01D-1521-08.1.fasta,77,TCGA-21-1078-01A-01D-1521-08,8CF9B32D-3D6F-4898-8C7A-89511B754021,TCGA-21-1078,0ab8d063-62b4-4d47-82aa-e3351a60029d,BAM,Aligned reads,Raw sequencing data,474,Lung Squamous Cell Carcinoma,NOT HISPANIC OR LATINO,WGS,MALE,TCGA-LUSC,Illumina HiSeq,Lung,WHITE,HG19_Broad_variant,TCGA-21-1078-01A,Primary Tumor,EFACAC80-9428-4482-9AEA-6B57FD172D49,Dead,s369__G13226.TCGA-21-1078-01A-01D-1521-08.1.bam
s10249,NaN,NaN,G14084.TCGA-22-1016-11A-01D-1521-08.3.bam,G14084.TCGA-22-1016-11A-01D-1521-08.3.fasta,65,TCGA-22-1016-11A-01D-1521-08,1845BC27-C3D3-4254-909C-2DC91F9E296B,TCGA-22-1016,037c57d1-b4a5-45dc-bda4-0550461d321b,BAM,Aligned reads,Raw sequencing data,822,Lung Squamous Cell Carcinoma,NOT HISPANIC OR LATINO,WGS,MALE,TCGA-LUSC,Illumina HiSeq,Lung,WHITE,HG19_Broad_variant,TCGA-22-1016-11A,Solid Tissue Normal,D06BC0A9-6053-454A-BDB9-BBDFDAA7C4E4,Dead,s380__G14084.TCGA-22-1016-11A-01D-1521-08.3.bam
s10250,NaN,NaN,G13232.TCGA-60-2726-01A-01D-1522-08.2.bam,G13232.TCGA-60-2726-01A-01D-1522-08.2.fasta,56,TCGA-60-2726-01A-01D-1522-08,A96EDDFC-3AFB-4BF8-A440-C91778113FBD,TCGA-60-2726,3d1f4059-2220-45b4-a4d2-b14f76cec96a,BAM,Aligned reads,Raw sequencing data,358,Lung Squamous Cell Carcinoma,NOT HISPANIC OR LATINO,WGS,MALE,TCGA-LUSC,Illumina HiSeq,Lung,WHITE,HG19_Broad_variant,TCGA-60-2726-01A,Primary Tumor,91A60807-2D0F-474E-B02F-18FEA788B319,Dead,s389__G13232.TCGA-60-2726-01A-01D-1522-08.2.bam
s10251,NaN,NaN,G14083.TCGA-21-1082-10B-01D-1521-08.4.bam,G14083.TCGA-21-1082-10B-01D-1521-08.4.fasta,61,TCGA-21-1082-10B-01D-1521-08,654AE6B0-65DA-4B11-B2FE-E7DD9C44E260,TCGA-21-1082,0e2ee54a-51c9-4868-842d-a2a1c1cfb016,BAM,Aligned reads,Raw sequencing data,NA,Lung Squamous Cell Carcinoma,NOT HISPANIC OR LATINO,WGS,MALE,TCGA-LUSC,Illumina HiSeq,Lung,WHITE,HG19_Broad_variant,TCGA-21-1082-10B,Blood Derived Normal,34E0475D-75C4-43C1-9715-027C19FA25F8,Alive,s399__G14083.TCGA-21-1082-10B-01D-1521-08.4.bam


In [12]:
# dfMeta.loc['s7010']
dfMeta.loc['s17588']

BarcodeSequence          NaN                                   
LinkerPrimerSequence     NaN                                   
filename                 G2185.TCGA-04-1371-11A-01D.7.bam      
filenames_new_suffix     G2185.TCGA-04-1371-11A-01D.7.fasta    
age_at_diagnosis         58                                    
aliquot_id               TCGA-04-1371-11A-01D-0516-08          
aliquot_uuid             81B30BC6-2BC4-48BA-AC99-E10786AFDC3A  
case_id                  TCGA-04-1371                          
case_uuid                bff84539-7862-45b2-b5fc-e77291fcca8b  
data_format              BAM                                   
data_subtype             Aligned reads                         
data_type                Raw sequencing data                   
days_to_death            NA                                    
disease_type             Ovarian Serous Cystadenocarcinoma     
ethnicity                Not available                         
experimental_strategy    WGS            

## Accounting for missing samples

In [13]:
print(np.setdiff1d(list(bactDataBarnDF.index), list(dfMeta.index)))
print(np.setdiff1d(list(virDataBarnDF.index), list(dfMeta.index)))
print(np.setdiff1d(list(vbDataBarnDF.index), list(dfMeta.index)),'\n')

print(np.setdiff1d(list(bactDataBarnDF.index), list(virDataBarnDF.index)),'\n')

print(np.setdiff1d(list(dfMeta.index), list(bactDataBarnDF.index)), len(np.setdiff1d(list(dfMeta.index), list(bactDataBarnDF.index))))
print(np.setdiff1d(list(dfMeta.index), list(virDataBarnDF.index)), len(np.setdiff1d(list(dfMeta.index), list(virDataBarnDF.index))))

['s2124' 's7807' 's8738' 's912']
['s2124' 's7807' 's8738' 's912']
['s2124' 's7807' 's8738' 's912'] 

['s17327' 's17330' 's17337' 's17339'] 

['s13044' 's13894' 's13930' 's13951' 's13987' 's13988' 's13990' 's14030'
 's14063' 's14095' 's14096' 's14097' 's14098' 's14099' 's14142' 's14175'
 's14176' 's14177' 's14224' 's14225' 's14228' 's14267' 's14312' 's14313'
 's14374' 's14743' 's17172' 's17553' 's17555' 's18092'] 30
['s13044' 's13894' 's13930' 's13951' 's13987' 's13988' 's13990' 's14030'
 's14063' 's14095' 's14096' 's14097' 's14098' 's14099' 's14142' 's14175'
 's14176' 's14177' 's14224' 's14225' 's14228' 's14267' 's14312' 's14313'
 's14374' 's14743' 's17172' 's17327' 's17330' 's17337' 's17339' 's17553'
 's17555' 's18092'] 34


In [14]:
missing = np.setdiff1d(list(dfMeta.index), list(vbDataBarnDF.index))
print(len(missing))
len(missing) == (dfMeta.shape[0] - vbDataBarnDF.shape[0])

34


False

In [15]:
print(sorted(missing))

['s13044', 's13894', 's13930', 's13951', 's13987', 's13988', 's13990', 's14030', 's14063', 's14095', 's14096', 's14097', 's14098', 's14099', 's14142', 's14175', 's14176', 's14177', 's14224', 's14225', 's14228', 's14267', 's14312', 's14313', 's14374', 's14743', 's17172', 's17327', 's17330', 's17337', 's17339', 's17553', 's17555', 's18092']


In [16]:
dfMeta.loc[sorted(missing),]

,BarcodeSequence,LinkerPrimerSequence,filename,filenames_new_suffix,age_at_diagnosis,aliquot_id,aliquot_uuid,case_id,case_uuid,data_format,data_subtype,data_type,days_to_death,disease_type,ethnicity,experimental_strategy,gender,investigation,platform,primary_site,race,reference_genome,sample_id,sample_type,sample_uuid,vital_status,Description
#SampleID,,,,,,,,,,,,,,,,,,,,,,,,,,,
s13044,NaN,NaN,TCGA-QG-A5YX-01A-11D-A28G-10_wgs_Illumina.bam,TCGA-QG-A5YX-01A-11D-A28G-10_wgs_Illumina.fasta,61,TCGA-QG-A5YX-01A-11D-A28G-10,10AD692B-4C3D-42DE-9B5E-4968441388B3,TCGA-QG-A5YX,300fff5a-0f61-402d-b845-3938ba7b1294,BAM,Aligned reads,Raw sequencing data,NA,Colon Adenocarcinoma,NOT HISPANIC OR LATINO,WGS,FEMALE,TCGA-COAD,Illumina HiSeq,Colorectal,BLACK OR AFRICAN AMERICAN,GRCh37-lite,TCGA-QG-A5YX-01A,Primary Tumor,346253C0-8E8B-459B-A6D5-03DA9586578F,Alive,s6238__TCGA-QG-A5YX-01A-11D-A28G-10_wgs_Illumina.bam
s13894,NaN,NaN,G78484.TCGA-26-A7UX-01B-11R-A39C-08.1.bam,G78484.TCGA-26-A7UX-01B-11R-A39C-08.1.fasta,47,TCGA-26-A7UX-01B-11R-A39C-08,B7B97BFF-DC3B-4E90-937A-9F58CD443AC9,TCGA-26-A7UX,5f9baf9b-6d2d-495a-83a1-77f7e4cb9ccd,BAM,Aligned reads,Raw sequencing data,NA,Glioblastoma Multiforme,NOT HISPANIC OR LATINO,RNA-Seq,MALE,TCGA-GBM,Illumina HiSeq,Brain,BLACK OR AFRICAN AMERICAN,HG19_Broad_variant,TCGA-26-A7UX-01B,Primary Tumor,6FBD4E97-943F-48C5-A869-7CE6B08BF576,Alive,s1383__G78484.TCGA-26-A7UX-01B-11R-A39C-08.1.bam
s13930,NaN,NaN,G67451.TCGA-RR-A6KB-01A-12R-A344-08.2.bam,G67451.TCGA-RR-A6KB-01A-12R-A344-08.2.fasta,51,TCGA-RR-A6KB-01A-12R-A344-08,580AFEC9-68A7-402D-8C83-42BE721F816E,TCGA-RR-A6KB,6f1b1011-d7a4-4a05-b6ff-288183e7254b,BAM,Aligned reads,Raw sequencing data,NA,Glioblastoma Multiforme,NOT HISPANIC OR LATINO,RNA-Seq,MALE,TCGA-GBM,Illumina HiSeq,Brain,BLACK OR AFRICAN AMERICAN,HG19_Broad_variant,TCGA-RR-A6KB-01A,Primary Tumor,3555B69D-2783-4811-9312-2B77C97500F5,Alive,s1360__G67451.TCGA-RR-A6KB-01A-12R-A344-08.2.bam
s13951,NaN,NaN,G78484.TCGA-4W-AA9R-01A-11R-A39C-08.1.bam,G78484.TCGA-4W-AA9R-01A-11R-A39C-08.1.fasta,61,TCGA-4W-AA9R-01A-11R-A39C-08,451E8218-8C5A-4E3E-96D7-CEB5EB15DA11,TCGA-4W-AA9R,f55a023c-84b8-4f7d-87db-7a9939733c59,BAM,Aligned reads,Raw sequencing data,NA,Glioblastoma Multiforme,NOT HISPANIC OR LATINO,RNA-Seq,MALE,TCGA-GBM,Illumina HiSeq,Brain,BLACK OR AFRICAN AMERICAN,HG19_Broad_variant,TCGA-4W-AA9R-01A,Primary Tumor,8BFCDBEF-692A-4CA8-BFDC-AFE7AE85ACFF,Alive,s1361__G78484.TCGA-4W-AA9R-01A-11R-A39C-08.1.bam
s13987,NaN,NaN,G78484.TCGA-06-A7TK-01A-21R-A39C-08.1.bam,G78484.TCGA-06-A7TK-01A-21R-A39C-08.1.fasta,64,TCGA-06-A7TK-01A-21R-A39C-08,F50A7996-B135-4925-A65B-6E74CB088E53,TCGA-06-A7TK,41685c5a-a548-483a-8a20-305ad8d61771,BAM,Aligned reads,Raw sequencing data,NA,Glioblastoma Multiforme,NOT HISPANIC OR LATINO,RNA-Seq,MALE,TCGA-GBM,Illumina HiSeq,Brain,BLACK OR AFRICAN AMERICAN,HG19_Broad_variant,TCGA-06-A7TK-01A,Primary Tumor,5012C42C-3FBB-4E0A-ABBB-78DCC6E03D9A,Alive,s1355__G78484.TCGA-06-A7TK-01A-21R-A39C-08.1.bam
s13988,NaN,NaN,G78484.TCGA-4W-AA9S-01A-11R-A39C-08.1.bam,G78484.TCGA-4W-AA9S-01A-11R-A39C-08.1.fasta,69,TCGA-4W-AA9S-01A-11R-A39C-08,B8F75FB0-8D56-4112-9FF5-5018DFC3C496,TCGA-4W-AA9S,7a8d2e13-315e-4512-85ce-b018d11c3bd3,BAM,Aligned reads,Raw sequencing data,NA,Glioblastoma Multiforme,NOT HISPANIC OR LATINO,RNA-Seq,MALE,TCGA-GBM,Illumina HiSeq,Brain,BLACK OR AFRICAN AMERICAN,HG19_Broad_variant,TCGA-4W-AA9S-01A,Primary Tumor,D0DE44E5-E959-4AEE-BB04-16A65FD4C0EE,Alive,s1392__G78484.TCGA-4W-AA9S-01A-11R-A39C-08.1.bam
s13990,NaN,NaN,TCGA-06-0137-10A-01D-0513-10_SOLiD.bam,TCGA-06-0137-10A-01D-0513-10_SOLiD.fasta,63,TCGA-06-0137-10A-01D-0513-10,BC2C55F9-A89B-4C45-BE1F-4F1CACB710C3,TCGA-06-0137,d0de6676-6ba1-4d79-a9b0-ec3f1e8a8775,BAM,Aligned reads,Raw sequencing data,812,Glioblastoma Multiforme,NOT HISPANIC OR LATINO,WGS,FEMALE,TCGA-GBM,ABI SOLiD,Brain,WHITE,NCBI36_BCM_variant,TCGA-06-0137-10A,Blood Derived Normal,8A7A84E1-C6BA-4E4C-BC48-695E8D79CFBA,Dead,s1526__TCGA-06-0137-10A-01D-0513-10_SOLiD.bam
s14030,N

In [17]:
missingDataVsMeta = np.setdiff1d(list(vbDataBarnDF.index), list(dfMeta.index))
vbDataBarnDFReconciled = vbDataBarnDF.drop(missingDataVsMeta)
vbDataBarnDFReconciled.shape

(18116, 1993)

In [18]:
dfMetaReconciled = dfMeta.loc[vbDataBarnDFReconciled.index]
dfMetaReconciled.shape

(18116, 27)

# Creating a merged metadata file

Data was taken from CGC SBGenomics website to align filenames with their gdc_file_uuid, which is how the 3D patient metadata dictionary is set up to be read.

In [19]:
cgcDataminer = pd.read_csv('cgcDataminerTotalNoDuplicates.csv')
originalMappingFile = pd.read_csv('cgc_qiime_mapping_file_18154_files.csv')
cgcDataminer.head(2)

,filename,size,gdc_file_uuid
0,ff361cc3e22a9619323ddbddf3aac564.bam,127.7 GB,D104749E-66ED-4722-98A5-5D59447E5779
1,ff180684131876b77d5fa19e93aaeb1e.bam,131.5 GB,62A7D6F2-3B71-4449-83FE-DA6BEEA65873


During the webscraping, filenames that were too long to display were interrupted with a "..." midway through the text. This caused a significant issue when trying to subsequently merge tables on filename, as 1028 filenames didn't match due to the ellipsis. The code below searches for the filenames with the artificial "...", searches for the full name in the main qiime metadata file, and replaces it accordingly.

In [20]:
count=0
partialString = ''
fullString = ''
for ii in range(len(cgcDataminer)):
    if "..." in cgcDataminer.iloc[ii]['filename']:
        partialString = cgcDataminer.iloc[ii]['filename']
        splitPartialString = partialString.split('...')[0]
        fullString = originalMappingFile[originalMappingFile['filename'].str.contains(splitPartialString)]['filename'].values[0]
        # Replace
        cgcDataminer.iloc[ii]['filename'] = fullString
        
        count+=1
        
print(count)

1028


In [21]:
# Sanity check to show that the filename replacement code above worked (should have 0 missing values)
missing = np.setdiff1d(list(cgcDataminer.filename), list(originalMappingFile.filename))
print(len(missing))

0


In [22]:
dfMetaReconciled.insert(loc=0, column='sampleID', value=dfMetaReconciled.index)  # Extract sampleID so it's not lost in the PD merging
dfMetaReconciled.head(2)

,sampleID,BarcodeSequence,LinkerPrimerSequence,filename,filenames_new_suffix,age_at_diagnosis,aliquot_id,aliquot_uuid,case_id,case_uuid,data_format,data_subtype,data_type,days_to_death,disease_type,ethnicity,experimental_strategy,gender,investigation,platform,primary_site,race,reference_genome,sample_id,sample_type,sample_uuid,vital_status,Description
s17489,s17489,NaN,NaN,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.fasta,51,TCGA-09-1659-01B-01R-1564-13,32D5209E-2EF2-4EF0-B902-585542D7D16B,TCGA-09-1659,644a88a7-dc56-468a-af5c-60278aab7642,BAM,Aligned reads,Raw sequencing data,304,Ovarian Serous Cystadenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,TCGA-09-1659-01B,Primary Tumor,01A6F7CD-10CC-4F33-AA75-B29ABD6C2EDB,Dead,s1570__TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam
s17512,s17512,NaN,NaN,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.fasta,36,TCGA-24-1105-01A-01R-1565-13,FD57ADC4-669C-4916-A2F5-E5F92CB0ADB6,TCGA-24-1105,7e01f23a-0b0e-46a2-9f30-c3a9d2c779d2,BAM,Aligned reads,Raw sequencing data,1442,Ovarian Serous Cystadenocarcinoma,Not available,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,TCGA-24-1105-01A,Primary Tumor,AE4AA065-8B1D-42A3-98EF-357E19C1F1E1,Dead,s1911__TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam


In [23]:
dfMetaReconciledUUID = pd.merge(dfMetaReconciled, cgcDataminer, how = 'left', on='filename')
print(dfMetaReconciledUUID.shape)
dfMetaReconciledUUID.head(3)

(18116, 30)


,sampleID,BarcodeSequence,LinkerPrimerSequence,filename,filenames_new_suffix,age_at_diagnosis,aliquot_id,aliquot_uuid,case_id,case_uuid,data_format,data_subtype,data_type,days_to_death,disease_type,ethnicity,experimental_strategy,gender,investigation,platform,primary_site,race,reference_genome,sample_id,sample_type,sample_uuid,vital_status,Description,size,gdc_file_uuid
0,s17489,NaN,NaN,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.fasta,51,TCGA-09-1659-01B-01R-1564-13,32D5209E-2EF2-4EF0-B902-585542D7D16B,TCGA-09-1659,644a88a7-dc56-468a-af5c-60278aab7642,BAM,Aligned reads,Raw sequencing data,304,Ovarian Serous Cystadenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,TCGA-09-1659-01B,Primary Tumor,01A6F7CD-10CC-4F33-AA75-B29ABD6C2EDB,Dead,s1570__TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam,14.6 GB,C38F96A6-0586-4F5E-B3E8-ADBB0358D9DC
1,s17512,NaN,NaN,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.fasta,36,TCGA-24-1105-01A-01R-1565-13,FD57ADC4-669C-4916-A2F5-E5F92CB0ADB6,TCGA-24-1105,7e01f23a-0b0e-46a2-9f30-c3a9d2c779d2,BAM,Aligned reads,Raw sequencing data,1442,Ovarian Serous Cystadenocarcinoma,Not available,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,TCGA-24-1105-01A,Primary Tumor,AE4AA065-8B1D-42A3-98EF-357E19C1F1E1,Dead,s1911__TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam,13.1 GB,7B59A22A-5799-4413-BB29-0E79643C540E
2,s17498,NaN,NaN,TCGA-13-1498-01A-01R-1565-13_rnaseq.bam,TCGA-13-1498-01A-01R-1565-13_rnaseq.fasta,73,TCGA-13-1498-01A-01R-1565-13,176D1CA2-EB1D-4F49-9B10-C129C822FA5A,TCGA-13-1498,56a30462-2819-4c18-95be-8e73880a4921,BAM,Aligned reads,Raw sequencing data,NA,Ovarian Serous Cystadenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,NCBI36_BCCAGSC_variant,TCGA-13-1498-01A,Primary Tumor,B38F209A-000B-4CBF-A4BD-0BC6F027FCE3,Alive,s1750__TCGA-13-1498-01A-01R-1565-13_rnaseq.bam,13.1 GB,53B17871-CE7E-4BD1-B4B5-A7D4D2003416


In [24]:
# Sanity check: All should have the same counts
tmp = dfMetaReconciledUUID[['filename','sampleID','gdc_file_uuid']]
tmp.count(axis=0)

filename         18116
sampleID         18116
gdc_file_uuid    18116
dtype: int64

In [25]:
dfMetaReconciledUUID.set_index('gdc_file_uuid', inplace=True)
dfMetaReconciledUUID.head(2)

,sampleID,BarcodeSequence,LinkerPrimerSequence,filename,filenames_new_suffix,age_at_diagnosis,aliquot_id,aliquot_uuid,case_id,case_uuid,data_format,data_subtype,data_type,days_to_death,disease_type,ethnicity,experimental_strategy,gender,investigation,platform,primary_site,race,reference_genome,sample_id,sample_type,sample_uuid,vital_status,Description,size
gdc_file_uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C38F96A6-0586-4F5E-B3E8-ADBB0358D9DC,s17489,NaN,NaN,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.fasta,51,TCGA-09-1659-01B-01R-1564-13,32D5209E-2EF2-4EF0-B902-585542D7D16B,TCGA-09-1659,644a88a7-dc56-468a-af5c-60278aab7642,BAM,Aligned reads,Raw sequencing data,304,Ovarian Serous Cystadenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,TCGA-09-1659-01B,Primary Tumor,01A6F7CD-10CC-4F33-AA75-B29ABD6C2EDB,Dead,s1570__TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam,14.6 GB
7B59A22A-5799-4413-BB29-0E79643C540E,s17512,NaN,NaN,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.fasta,36,TCGA-24-1105-01A-01R-1565-13,FD57ADC4-669C-4916-A2F5-E5F92CB0ADB6,TCGA-24-1105,7e01f23a-0b0e-46a2-9f30-c3a9d2c779d2,BAM,Aligned reads,Raw sequencing data,1442,Ovarian Serous Cystadenocarcinoma,Not available,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,TCGA-24-1105-01A,Primary Tumor,AE4AA065-8B1D-42A3-98EF-357E19C1F1E1,Dead,s1911__TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam,13.1 GB


## QIIME metadata cleaning

In [26]:
cols_to_remove = list(['size',
                       'BarcodeSequence',
                       'LinkerPrimerSequence',
                       'filenames_new_suffix',
                       'aliquot_id',
                       'case_id',
                       'data_format',
                       'data_subtype',
                       'data_type',
                       'sample_id',
                       'Description'])

keyMetaReconciledUUID = dfMetaReconciledUUID.drop(labels = cols_to_remove, axis = 'columns')
keyMetaReconciledUUID.head(2)

,sampleID,filename,age_at_diagnosis,aliquot_uuid,case_uuid,days_to_death,disease_type,ethnicity,experimental_strategy,gender,investigation,platform,primary_site,race,reference_genome,sample_type,sample_uuid,vital_status
gdc_file_uuid,,,,,,,,,,,,,,,,,,
C38F96A6-0586-4F5E-B3E8-ADBB0358D9DC,s17489,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam,51,32D5209E-2EF2-4EF0-B902-585542D7D16B,644a88a7-dc56-468a-af5c-60278aab7642,304,Ovarian Serous Cystadenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,Primary Tumor,01A6F7CD-10CC-4F33-AA75-B29ABD6C2EDB,Dead
7B59A22A-5799-4413-BB29-0E79643C540E,s17512,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam,36,FD57ADC4-669C-4916-A2F5-E5F92CB0ADB6,7e01f23a-0b0e-46a2-9f30-c3a9d2c779d2,1442,Ovarian Serous Cystadenocarcinoma,Not available,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,Primary Tumor,AE4AA065-8B1D-42A3-98EF-357E19C1F1E1,Dead


## Extracting static data from Pickle file into keyMetaUUID

In [27]:
staticMetaPickle = list([
    'tissue_source_site_label',
    'data_submitting_center_label',
    'country_of_sample_procurement',
    'histological_diagnosis_label',
    'pathologic_t_label',
    'pathologic_n_label',
    'pathologic_stage_label',
    'icd03_histology_label',
    'icd03_histology_site',
    'icd10',
    'portion_is_ffpe',
    'new_tumor_event_after_initial_trtmt',
    'primary_therapy_outcome_success_label',
    'portion_weight',
    'aliquot_concentration',
    'analyte_A260A280Ratio',
    'analyte_amount',
    'analyte_type_label',
    'radiation_therapy_code_label',
    'radiation_therapy_site_label',
    'radiation_therapy_type_label',
    'year_of_diagnosis',
    'vital_status_label'
])

appended_data = []
appended_ind = []
for ii in range(len(keyMetaReconciledUUID)):
    file_uuid = keyMetaReconciledUUID.index[ii]
    appended_ind.append(file_uuid)
    dfHolder = fileNameTesting[file_uuid]
    appended_data.append(pd.DataFrame(dfHolder[staticMetaPickle].iloc[0]))

# Concatenate extracted data into Dataframe object
extractedStaticPickleDF = pd.concat(appended_data, axis=1).transpose()
extractedStaticPickleDF['gdc_file_uuid'] = appended_ind
extractedStaticPickleDF.set_index('gdc_file_uuid', inplace=True)
print(extractedStaticPickleDF.shape)
extractedStaticPickleDF.head()

(18116, 23)


,tissue_source_site_label,data_submitting_center_label,country_of_sample_procurement,histological_diagnosis_label,pathologic_t_label,pathologic_n_label,pathologic_stage_label,icd03_histology_label,icd03_histology_site,icd10,portion_is_ffpe,new_tumor_event_after_initial_trtmt,primary_therapy_outcome_success_label,portion_weight,aliquot_concentration,analyte_A260A280Ratio,analyte_amount,analyte_type_label,radiation_therapy_code_label,radiation_therapy_site_label,radiation_therapy_type_label,year_of_diagnosis,vital_status_label
gdc_file_uuid,,,,,,,,,,,,,,,,,,,,,,,
C38F96A6-0586-4F5E-B3E8-ADBB0358D9DC,UCSF,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,110,0.16,1.8,269,RNA,NaN,NaN,NaN,1997,Dead
7B59A22A-5799-4413-BB29-0E79643C540E,Washington University,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,130,0.15,1.8,236,RNA,06461927-0E16-4D01-B765-C5F76574560D,Distant Recurrence,EXTERNAL BEAM,2001,Dead
53B17871-CE7E-4BD1-B4B5-A7D4D2003416,Memorial Sloan Kettering,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,131,0.16,1.8,392,RNA,NaN,NaN,NaN,2006,Alive
B98E97B9-6C7A-437E-9724-4D76D5FEFC06,Duke,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,104,0.16,1.8,53.7,RNA,NaN,NaN,NaN,2005,Dead
E9EC871C-163C-4FD8-AE67-02BA37605467,Duke,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,147,0.17,1.8,117,RNA,NaN,NaN,NaN,2008,Alive


## Merge extracted data into one main metadata dataframe

In [28]:
tcgaMetadataPickledReconciled = pd.merge(keyMetaReconciledUUID,extractedStaticPickleDF,how='left',left_index=True,right_index=True)
print(keyMetaReconciledUUID.shape)
print(extractedStaticPickleDF.shape)
print(tcgaMetadataPickledReconciled.shape)
tcgaMetadataPickledReconciled.head()

(18116, 18)
(18116, 23)
(18116, 41)


,sampleID,filename,age_at_diagnosis,aliquot_uuid,case_uuid,days_to_death,disease_type,ethnicity,experimental_strategy,gender,investigation,platform,primary_site,race,reference_genome,sample_type,sample_uuid,vital_status,tissue_source_site_label,data_submitting_center_label,country_of_sample_procurement,histological_diagnosis_label,pathologic_t_label,pathologic_n_label,pathologic_stage_label,icd03_histology_label,icd03_histology_site,icd10,portion_is_ffpe,new_tumor_event_after_initial_trtmt,primary_therapy_outcome_success_label,portion_weight,aliquot_concentration,analyte_A260A280Ratio,analyte_amount,analyte_type_label,radiation_therapy_code_label,radiation_therapy_site_label,radiation_therapy_type_label,year_of_diagnosis,vital_status_label
gdc_file_uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C38F96A6-0586-4F5E-B3E8-ADBB0358D9DC,s17489,TCGA-09-1659-01B-01R-1564-13_GRCh37-lite_rnaseq.bam,51,32D5209E-2EF2-4EF0-B902-585542D7D16B,644a88a7-dc56-468a-af5c-60278aab7642,304,Ovarian Serous Cystadenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,Primary Tumor,01A6F7CD-10CC-4F33-AA75-B29ABD6C2EDB,Dead,UCSF,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,110,0.16,1.8,269,RNA,NaN,NaN,NaN,1997,Dead
7B59A22A-5799-4413-BB29-0E79643C540E,s17512,TCGA-24-1105-01A-01R-1565-13_GRCh37-lite_rnaseq.bam,36,FD57ADC4-669C-4916-A2F5-E5F92CB0ADB6,7e01f23a-0b0e-46a2-9f30-c3a9d2c779d2,1442,Ovarian Serous Cystadenocarcinoma,Not available,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,Primary Tumor,AE4AA065-8B1D-42A3-98EF-357E19C1F1E1,Dead,Washington University,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,130,0.15,1.8,236,RNA,06461927-0E16-4D01-B765-C5F76574560D,Distant Recurrence,EXTERNAL BEAM,2001,Dead
53B17871-CE7E-4BD1-B4B5-A7D4D2003416,s17498,TCGA-13-1498-01A-01R-1565-13_rnaseq.bam,73,176D1CA2-EB1D-4F49-9B10-C129C822FA5A,56a30462-2819-4c18-95be-8e73880a4921,NA,Ovarian Serous Cystadenocarcinoma,NOT HISPANIC OR LATINO,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,NCBI36_BCCAGSC_variant,Primary Tumor,B38F209A-000B-4CBF-A4BD-0BC6F027FCE3,Alive,Memorial Sloan Kettering,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,131,0.16,1.8,392,RNA,NaN,NaN,NaN,2006,Alive
B98E97B9-6C7A-437E-9724-4D76D5FEFC06,s17528,TCGA-29-1705-02A-01R-1567-13_GRCh37-lite_rnaseq.bam,47,CC3AFD8C-3639-437F-A22B-AADC9F6C5AAA,f9c835db-2ab6-4bf5-826f-48723493c0ec,555,Ovarian Serous Cystadenocarcinoma,Not available,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,Recurrent Tumor,B5163587-A94D-4674-9F11-76A5ADC9151D,Dead,Duke,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,104,0.16,1.8,53.7,RNA,NaN,NaN,NaN,2005,Dead
E9EC871C-163C-4FD8-AE67-02BA37605467,s17535,TCGA-29-1777-01A-01R-1567-13_GRCh37-lite_rnaseq.bam,47,CBEBA637-1A32-45F1-83F3-614725665436,5ced5b58-fab4-4bcd-b706-d6e49ce0cfc5,NA,Ovarian Serous Cystadenocarcinoma,Not available,RNA-Seq,FEMALE,TCGA-OV,Illumina HiSeq,Ovary,WHITE,GRCh37-lite,Primary Tumor,AFB43292-69C2-4D84-84F5-0BDE9A6E9EA5,Alive,Duke,Canada's Michael Smith Genome Sciences Centre,None,Serous Cystadenocarcinoma,Not available,Not available,Not available,8441/3,C56.9,C56.9,NO,NaN,Not available,147,0.17,1.8,117,RNA,NaN,NaN,NaN,2008,Alive


In [29]:
tcgaMetadataPickledReconciled.radiation_therapy_code_label.value_counts()

4F455D1B-8492-4C47-BC16-01DE2AC1898A    10
CF5E2AC7-1264-4B95-86C2-6C3748C3F63E    9 
B55C8E7E-F2D6-452F-9D89-E51054CFACA4    8 
79AB7E62-6F27-4A0B-BD9D-6604C379142E    7 
0170BB9D-3F24-4932-92EE-9BAA1D3AE3DC    7 
636B9443-3E2C-48BD-AE5B-4830A28CDEE5    7 
C89EC5A7-544E-49E0-9CD1-C0D2C31EA305    7 
0D9A7590-E13D-4EE4-8799-90929FC8818F    7 
E4E00581-06DC-4638-B03A-43CCCDD89A46    7 
F92B3052-5815-4AFE-85DD-80ACEC4563A4    7 
9939E1CA-953B-4E0A-A861-225FEE9920C8    7 
64AF2B41-56BC-4089-8B6F-3DBE74A9273A    7 
EED3A9E7-41B1-468D-A4D4-62275344027C    7 
597CFDF4-47EE-4CFD-8D2B-66BD1D75DE2B    7 
DC4F3B20-A952-41B7-B484-E35D5B0FB589    6 
659B8763-1226-49D0-8ED4-E5BDB14548E6    6 
F747E175-7B33-42B7-A077-B3581BBCF094    6 
1A7DD856-99F0-40D6-90A7-1BA2202515F0    6 
ED646923-A759-4AE3-A882-C6928015194F    6 
F3DA80CD-337F-4782-90F1-F7D94AA9A17C    6 
B1912257-7FD6-4E0B-9609-96C0BFAB93A9    6 
133ADBF7-0275-4A02-8BB4-5EC6F14E1AD9    6 
DF9E861E-D0E8-4C2D-808B-352783005D25    6 
873291FF-F5

# Exporting data to csv for manipulation in R

In [30]:
# vbBarnDF.to_csv('vbBarnDF_all_cancers.csv')
# vbBarnDFFilt.to_csv('vbBarnFilt_4_cancers.csv')
vbDataBarnDFReconciled.to_csv('vbDataBarnDFReconciled_18116_files.csv')
tcgaMetadataPickledReconciled.to_csv('tcgaMetadataPickledReconciled_18116_files.csv')